### Saha for N Ionization

The ionization energies are obtained from [NIST](https://physics.nist.gov/cgi-bin/ASD/ie.pl?spectra=Xe&units=1&at_num_out=on&el_name_out=on&seq_out=on&shells_out=on&level_out=on&e_out=0&unc_out=on&biblio=on)

The statistical weights are approximately derived also with NIST, using g_i = 2 J_i + 1 of the ground level of that state


In [1]:
import numpy as np
from hnc.hnc.constants import *

from saha.core.saha import plasma, calculate_ionization_fractions

from saha.core.table_generator import saha_table

import matplotlib.pyplot as plt

plt.rcParams.update({'font.size': 14})
plt.rcParams["figure.figsize"] = (6,4)
plt.rcParams["figure.dpi"] = 200
plt.rcParams['xtick.labelsize']=14

color_smooth = lambda N: plt.cm.viridis(np.linspace(0,1,N))
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [37]:
Ar_ionization_energies_AU = np.array([0, 15.7596119, 27.62967, 40.735, 59.58, 74.84, 91.290])*eV_to_AU
Ar_J_ground_level = np.array([0, 3/2, 2, 3/2, 0, 1/2, 0])
Ar_ionization_degeneracies = 2*Ar_J_ground_level + 1
# Ar_ionization_degeneracies = np.array([1, 5.83, 10.9])
print("Using degeneracies of: ", Ar_ionization_degeneracies )


# Input: Number of ionizations
Z = 18

nn_invcc_at_Pbar_TK = lambda Pbar, TK: Pbar*bar_to_AU/(TK*K_to_AU)*AU_to_invcc
Ar_nn_invcc = nn_invcc_at_Pbar_TK(25, 290)
Ar_TK_peak = 1.5*eV_to_K#17.761029411764707e3 # 0.008097165991902834 ns?

nn_invcc = Ar_nn_invcc
nn_AU = nn_invcc  * invcc_to_AU
T_AU  = Ar_TK_peak*K_to_AU

Ar_plasma = plasma("Ar25bar_weak-coupling_Δχ", Z, Ar_ionization_energies_AU, Ar_ionization_degeneracies)

Using degeneracies of:  [1. 4. 5. 4. 1. 2. 1.]


In [38]:
Zbar, xi_array, χ_array  = calculate_ionization_fractions(Ar_plasma, nn_AU, T_AU, IPD=True, N_ions=2)

In [39]:
print(f"<Z> = {Zbar}")
print(f"xi = {xi_array}")
print(f"χ = {χ_array}")

<Z> = 0.055216204534848594
xi = [9.44784345e-01 5.52151049e-02 5.49792579e-07]
χ = [0.         0.55121857 0.96543869]


### Daniels Check

In [27]:
def λD(T):
    return np.sqrt(2*π/(m_e*T))

λD(T_AU)*AU_to_cm, 2*λD(T_AU)**-3/nn_AU, 2*λD(T_AU)**-3/nn_AU/Zbar

(5.6496396998645206e-08, 17.762689329748497, 262.0807160709689)

In [6]:
2*λD(T_AU)**-3/nn_AU/Zbar * Ar_ionization_degeneracies[2]/Ar_ionization_degeneracies[1]

471.97662788458956

In [7]:
def saha_equation(ne, T, Δε, degeneracy_ratio):
    # Saha equation in atomic units
    saha_ratio = degeneracy_ratio * 2*λD(T)**-3 * np.exp(- Δε / T) / ne
    return saha_ratio, 


In [8]:
saha_equation(nn_AU, T_AU, Ar_ionization_energies_AU[2]-Ar_ionization_energies_AU[1] , Ar_ionization_degeneracies[2]/Ar_ionization_degeneracies[1]), 17.1 * 5.83/1 * 3.77e-4/1, 17.1 * 10.9/5.83 * 1e-6/3.77e-4

((0.009802897732147984,), 0.03758426100000001, 0.08480329039860594)

In [9]:
xi_D = np.array([0.176, 3.18e-5])
(xi_D[0] + 2*xi_D[1])*xi_D[1]/xi_D[0]


3.181149136363637e-05